In [36]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
!pip install transformers
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [37]:
file_path = "/kaggle/input/cloths/renttherunway_final_data.json"
data = pd.read_json(file_path, lines=True)
df = pd.DataFrame(data)

In [38]:
df.dropna(inplace=True)
filtered_categories = df['category'].value_counts()[df['category'].value_counts() >= 1000]
df = df[df['category'].isin(filtered_categories.index)]
filtered_categories = df['rented for'].value_counts()[df['rented for'].value_counts() >= 1000]
df = df[df['rented for'].isin(filtered_categories.index)]
unique_values_per_column = df.nunique()
print(unique_values_per_column)
df.info()

fit                    3
user_id            76571
bust size            101
item_id             4937
weight               186
rating                 5
rented for             8
review_text       138919
body type              7
review_summary    114494
category              11
height                24
size                  56
age                   82
review_date         2239
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Index: 139821 entries, 0 to 192543
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   fit             139821 non-null  object 
 1   user_id         139821 non-null  int64  
 2   bust size       139821 non-null  object 
 3   item_id         139821 non-null  int64  
 4   weight          139821 non-null  object 
 5   rating          139821 non-null  float64
 6   rented for      139821 non-null  object 
 7   review_text     139821 non-null  object 
 8   body type       139821 non-null  object 


In [39]:
df.iloc[0]

fit                                                             fit
user_id                                                      420272
bust size                                                       34d
item_id                                                     2260466
weight                                                       137lbs
rating                                                         10.0
rented for                                                 vacation
review_text       An adorable romper! Belt and zipper were a lit...
body type                                                 hourglass
review_summary                                 So many compliments!
category                                                     romper
height                                                        5' 8"
size                                                             14
age                                                            28.0
review_date                                     

In [40]:
df['height_inches'] = df['height'].apply(lambda x: int(x.split("'")[0]) * 12 + int(x.split("'")[1].replace('"', '')))
df['weight_lbs'] = df['weight'].apply(lambda x: int(x.replace('lbs', '')))
df['bust_size_int'] = df['bust size'].str[:2].astype(int)

product:
fit,bust size,rating, 

user:
age
size,height, vacation,bodytype,weight,

In [41]:
df['text_concat'] = df['review_text'] + ' ' + df['review_summary'] +' ' + df['category'] + ' ' + df['bust size'] + ' ' + df['body type'] + ' ' + df['rented for']
df['tokenized_text'] = df['text_concat'].apply(word_tokenize)
word2vec_model = Word2Vec(sentences=df['tokenized_text'], vector_size=100, window=5, min_count=1, workers=4)

In [42]:
# Function to get the embedding for an item_id
def get_item_embedding(item_id):
    text = df[df['item_id'] == item_id]['tokenized_text'].values[0]
    height = df[df['item_id'] == item_id]['height_inches'].mean()
    weight = df[df['item_id'] == item_id]['weight_lbs'].mean()
    age = df[df['item_id'] == item_id]['age'].mean()
    rating = df[df['item_id'] == item_id]['rating'].mean()
    size = df[df['item_id'] == item_id]['size'].mean()
    bust_size = df[df['item_id'] == item_id]['bust_size_int'].mean()
    
    embedding = []
    for word in text:
        if word in word2vec_model.wv:
            embedding.append(word2vec_model.wv[word])
    if embedding:
        word_embedding = np.mean(embedding, axis=0)
    else:
        word_embedding = np.zeros(100)  # Default zero vector if no words found in the model
        
    numerical_embedding = np.array([height, weight, age, rating])
#     numerical_embedding = np.array([size, rating])
    
    return np.hstack([word_embedding, numerical_embedding])


In [43]:
product_embeddings = {}
for item_id in df['item_id'].unique():
    product_embeddings[item_id] = get_item_embedding(item_id)

In [44]:
embedding_shapes = set()
for item_id in df['item_id'].unique():
    embedding_shapes.add(product_embeddings[item_id].shape)

# Print the set of shapes
print("Unique Embedding Shapes:", embedding_shapes)

Unique Embedding Shapes: {(104,)}


In [45]:
# Function to get the embedding for an item_id
def get_user_embedding(item_id):
    text = df[df['user_id'] == item_id]['tokenized_text'].values[0]
    height = df[df['user_id'] == item_id]['height_inches'].mean()
    weight = df[df['user_id'] == item_id]['weight_lbs'].mean()
    age = df[df['user_id'] == item_id]['age'].mean()
    bust_size = df[df['item_id'] == item_id]['bust_size_int'].mean()
#     rating = df[df['user_id'] == item_id]['rating'].mean()
    
    embedding = []
    for word in text:
        if word in word2vec_model.wv:
            embedding.append(word2vec_model.wv[word])
    if embedding:
        word_embedding = np.mean(embedding, axis=0)
    else:
        word_embedding = np.zeros(100)  # Default zero vector if no words found in the model
        
    numerical_embedding = np.array([height, weight, age,bust_size])
    
    return np.hstack([word_embedding, numerical_embedding])


In [46]:
user_embeddings = {}
for item_id in df['user_id'].unique():
    user_embeddings[item_id] = get_user_embedding(item_id)

In [47]:
embedding_shapes = set()
for item_id in df['user_id'].unique():
    embedding_shapes.add(user_embeddings[item_id].shape)

# Print the set of shapes
print("Unique Embedding Shapes:", embedding_shapes)

Unique Embedding Shapes: {(104,)}


In [48]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Concatenate

In [49]:
print(df)

        fit  user_id bust size  item_id  weight  rating     rented for  \
0       fit   420272       34d  2260466  137lbs    10.0       vacation   
1       fit   273551       34b   153475  132lbs    10.0          other   
3       fit   909926       34c   126335  135lbs     8.0  formal affair   
4       fit   151944       34b   616682  145lbs    10.0        wedding   
5       fit   734848       32b   364092  138lbs     8.0           date   
...     ...      ...       ...      ...     ...     ...            ...   
192539  fit    66386      34dd  2252812  140lbs    10.0           work   
192540  fit   118398       32c   682043  100lbs    10.0           work   
192541  fit    47002       36a   683251  135lbs     6.0       everyday   
192542  fit   961120       36c   126335  165lbs    10.0        wedding   
192543  fit   123612       36b   127865  155lbs    10.0        wedding   

                                              review_text          body type  \
0       An adorable romper! Bel

In [50]:
# Prepare training data (user_embeddings, item_embeddings, ratings)
X_user = np.array([user_embeddings[user_id] for user_id in df['user_id']])
X_item = np.array([product_embeddings[item_id] for item_id in df['item_id']])
X_train = np.concatenate((X_user, X_item), axis=1)
y_train = df['rating'].values

In [51]:
print(X_train.shape)

(139821, 208)


In [52]:



X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


model = Sequential([
    Dense(100, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(25, activation='relu'),
    Dense(1)  
])

model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=64)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: nan - val_loss: nan
Epoch 2/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 3/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 4/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 5/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 6/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: nan - val_loss: nan
Epoch 7/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 8/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 9/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 10/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 11/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 12/15
1748/1748 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: nan - val_loss: nan
Epoch 13/15
1

In [53]:
new_user_embedding=user_embeddings[420272]
item_ids = list(product_embeddings.keys())
item_embeddings_array = np.array([product_embeddings[item_id] for item_id in item_ids])

concatenated_embeddings = np.concatenate([np.tile(new_user_embedding, (len(item_ids), 1)), item_embeddings_array], axis=1)

predictions = model.predict(concatenated_embeddings)

predictions_with_ids = list(zip(item_ids, predictions.flatten()))

predictions_with_ids.sort(key=lambda x: x[1], reverse=True)


155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [54]:
top_indices = np.argsort(predictions.flatten())[-100:]

top_item_ids = [item_ids[i] for i in top_indices]

print("Top 10 Recommended Item IDs:", top_item_ids)

Top 10 Recommended Item IDs: [1513452, 1768942, 916623, 149913, 1414833, 2076492, 561215, 1310632, 1667586, 392822, 2015822, 2057094, 1434163, 1044125, 447098, 1074283, 2431951, 1515649, 618661, 1766932, 926842, 1863819, 1006741, 1613149, 1780765, 1558936, 1709498, 1038630, 2382109, 228702, 1592991, 2236220, 1003076, 1971222, 1138140, 561864, 2834871, 1831674, 1308013, 1251146, 451860, 149459, 664558, 1325194, 1223725, 1772024, 1549763, 1698815, 781281, 646029, 482477, 671761, 961819, 1762397, 325844, 1983744, 1227811, 987536, 1168767, 555311, 664127, 383730, 1769204, 341450, 2710114, 1804642, 1099646, 2363191, 1979624, 646512, 999526, 898529, 1617482, 1820237, 677837, 1331220, 377131, 598578, 308000, 2339359, 1437612, 571536, 1188264, 935316, 1446293, 956997, 1015626, 1068027, 2739219, 1388246, 393530, 799258, 2073553, 1859884, 2589683, 611674, 397391, 880935, 331924, 2751662]


In [55]:

filtered_recommendations = []
for item_id in top_item_ids:
    rented_for_category = df[df['item_id'] == item_id]['rented for'].iloc[0]
    if rented_for_category == 'wedding':
        filtered_recommendations.append(item_id)

print("Filtered Recommended Item IDs for weddings:", filtered_recommendations)


Filtered Recommended Item IDs for weddings: [1513452, 1667586, 1044125, 447098, 1074283, 618661, 926842, 1006741, 228702, 1003076, 1971222, 1308013, 1251146, 1549763, 646029, 482477, 671761, 1769204, 341450, 1979624, 1617482, 677837, 1331220, 1446293, 1015626, 393530, 397391, 880935]


In [56]:
a=df[df['user_id'] == 420272]
print(a)

          fit  user_id bust size  item_id  weight  rating rented for  \
0         fit   420272       34d  2260466  137lbs    10.0   vacation   
16494     fit   420272       34d  1083818  137lbs    10.0   vacation   
42588     fit   420272       34d  2340996  137lbs     2.0   everyday   
61172     fit   420272       34d  2363191  137lbs    10.0      party   
70351     fit   420272       34d   348662  137lbs     8.0      other   
135118  small   420272       34d  2431951  137lbs     6.0   vacation   

                                              review_text  body type  \
0       An adorable romper! Belt and zipper were a lit...  hourglass   
16494   Absolutely gorgeous dress. Great fit, no compl...  hourglass   
42588   I really was not into this jumpsuit. It has a ...  hourglass   
61172   SO many compliments. Glad I got the 6-- fit my...  hourglass   
70351   I could wear this literally forever. It is fai...  hourglass   
135118  I had to send this back in my unlimited shipme...  hour

In [70]:
a=df[df['item_id'] == 1513452]
print(a)

          fit  user_id bust size  item_id  weight  rating     rented for  \
4341      fit   630777       34d  1513452  125lbs     8.0        wedding   
16450     fit   215971      34dd  1513452  135lbs    10.0       everyday   
16855     fit   101458       34b  1513452  150lbs    10.0           date   
27808     fit   134103       36d  1513452  160lbs     6.0        wedding   
31952   small   722597      34dd  1513452  152lbs     6.0       vacation   
32111   small   970865       32d  1513452  135lbs     6.0          other   
42813     fit   375755       34b  1513452  130lbs     8.0        wedding   
65952     fit   911607       32d  1513452  135lbs    10.0           date   
79475     fit   530487       36c  1513452  140lbs    10.0          other   
107329  large   870573       32c  1513452  120lbs     6.0        wedding   
122768  large   428991       34c  1513452  120lbs     8.0       everyday   
127125  small   620835       36b  1513452  150lbs    10.0        wedding   
138135    fi

In [69]:
def get_new_user_embedding(new_user_data):
    text = new_user_data['tokenized_text']
    height = new_user_data['height_inches']
    weight = new_user_data['weight_lbs']
    age = new_user_data['age']
    bust_size = new_user_data['bust_size_int']
    
    embedding = []
    for word in text:
        if word in word2vec_model.wv:
            embedding.append(word2vec_model.wv[word])
    if embedding:
        word_embedding = np.mean(embedding, axis=0)
    else:
        word_embedding = np.zeros(100)  
        
    numerical_embedding = np.array([height, weight, age, bust_size])
    
    return np.hstack([word_embedding, numerical_embedding])

# Generate embedding for the new user
new_user_data = {
    'tokenized_text': ['wedding', 'hourglass'],  
    'height_inches': 65,  
    'weight_lbs': 150,    
    'age': 30,            
    'bust_size_int': 34  
}
new_user_embedding = get_new_user_embedding(new_user_data)

# Concatenate the new user's embedding with item embeddings
item_ids = list(product_embeddings.keys())
item_embeddings_array = np.array([product_embeddings[item_id] for item_id in item_ids])
concatenated_embeddings = np.concatenate([np.tile(new_user_embedding, (len(item_ids), 1)), item_embeddings_array], axis=1)

predictions = model.predict(concatenated_embeddings)

predictions_with_ids = list(zip(item_ids, predictions.flatten()))
predictions_with_ids.sort(key=lambda x: x[1], reverse=True)

top_indices = np.argsort(predictions.flatten())[-100:]

top_item_ids = [item_ids[i] for i in top_indices]

print("Top 10 Recommended Item IDs:", top_item_ids)

filtered_recommendations = []
for item_id in top_item_ids:
    rented_for_category = df[df['item_id'] == item_id]['rented for'].iloc[0]
    if rented_for_category == 'wedding':
        filtered_recommendations.append(item_id)

print("Filtered Recommended Item IDs for weddings:", filtered_recommendations)


155/155 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Top 10 Recommended Item IDs: [1513452, 1768942, 916623, 149913, 1414833, 2076492, 561215, 1310632, 1667586, 392822, 2015822, 2057094, 1434163, 1044125, 447098, 1074283, 2431951, 1515649, 618661, 1766932, 926842, 1863819, 1006741, 1613149, 1780765, 1558936, 1709498, 1038630, 2382109, 228702, 1592991, 2236220, 1003076, 1971222, 1138140, 561864, 2834871, 1831674, 1308013, 1251146, 451860, 149459, 664558, 1325194, 1223725, 1772024, 1549763, 1698815, 781281, 646029, 482477, 671761, 961819, 1762397, 325844, 1983744, 1227811, 987536, 1168767, 555311, 664127, 383730, 1769204, 341450, 2710114, 1804642, 1099646, 2363191, 1979624, 646512, 999526, 898529, 1617482, 1820237, 677837, 1331220, 377131, 598578, 308000, 2339359, 1437612, 571536, 1188264, 935316, 1446293, 956997, 1015626, 1068027, 2739219, 1388246, 393530, 799258, 2073553, 1859884, 2589683, 611674, 397391, 880935, 331924, 2751662]
Filtered Recommended Item IDs for weddings: [1513452, 1667586, 10441